In [1]:
from billions.data_loader import *
from billions.back_trader import *

In [2]:
fname = 'data/factor.csv'
cname = 'data/close.csv'
ftype = 'csv'
data = load(fname, cname, ftype)


In [43]:
factor = data[['factor']]
today_factor = factor.loc['2007-04-04', :].sort_values('factor', ascending = False).reset_index()
today_factor.head()

,trade_dt,code,factor
0,2007-04-04,CI005028,28
1,2007-04-04,CI005003,27
2,2007-04-04,CI005027,26
3,2007-04-04,CI005021,25
4,2007-04-04,CI005011,24


In [59]:
today_factor['factor'] = pd.qcut(today_factor['factor'], 5, labels=range(1, 6, 1))

['CI005016', 'CI005005', 'CI005010', 'CI005006', 'CI005002', 'CI005029']